In [1]:
import pandas as pd
import altair as alt
from altair import datum

## Read in hurricane tracts data

Specify that Season shouldn't be parsed as a number, even though it looks like that – keep as string

Change ISO_time to a true date and time data type

In [4]:
tracts = pd.read_excel('data/hurricane_tracts.xlsx', 
                       dtype={'Season':object})
tracts['ISO_time'] = pd.to_datetime(tracts.ISO_time)

/Users/emonson/anaconda3/envs/geo/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
tracts.head()

,Longitude,Latitude,Serial_Num,Season,Num,Basin,Sub_basin,Hurricane Name,ISO_time,Nature,Wind in knots,Pressure in millibars,Wind Percentile,Pressure Percentile,Track_type,State,SubRegion,Region,Country
0,-94.8,28.0,1851176N28265,1851,1,NaN,MM,1851176N28265,1851-06-25 00:00:00,TS,80,NaN,84.767,NaN,main,NaN,NaN,NaN,NaN
1,-95.4,28.0,1851176N28265,1851,1,NaN,MM,1851176N28265,1851-06-25 06:00:00,TS,80,NaN,84.767,NaN,main,NaN,NaN,NaN,NaN
2,-96.0,28.0,1851176N28265,1851,1,NaN,MM,1851176N28265,1851-06-25 12:00:00,TS,80,NaN,84.767,NaN,main,NaN,NaN,NaN,NaN
3,-96.5,28.1,1851176N28265,1851,1,NaN,MM,1851176N28265,1851-06-25 18:00:00,TS,80,NaN,84.767,NaN,main,NaN,NaN,NaN,NaN
4,-96.8,28.2,1851176N28265,1851,1,NaN,MM,1851176N28265,1851-06-25 21:00:00,TS,80,NaN,84.767,NaN,main,NaN,NaN,NaN,NaN


## MaxRowsError

Can take off limit for number of rows, but that's not a great idea because the notebook will have a Vega-Lite JSON specification (text) embedded for every output pane, which also includes the data, so you end up with huge notebooks!

Instead, we can specify that the outputs should just refer to a JSON file on your local drive and load in the data from there for each plot.

See the Altair tutorial notebook `03-Binning-and-aggregation` for more details.

In [10]:
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

## Selection linking

This particular interactive visualization combination taught me something new about this hurricane tracts data that I'd been using for years! 

**Notice the pressure and wind speed range for older measurements vs more modern ones!**

In [11]:
interval = alt.selection(type='interval', encodings=['x'])

timeline = alt.Chart(tracts).mark_line().encode(
    x=alt.X('ISO_time:T', timeUnit='year'),
    y='distinct(Serial_Num):Q'
).add_selection(
    interval
).properties(
    width=300,
    height=200
)

scatter = alt.Chart(tracts).mark_circle(opacity=0.3).encode(
    x=alt.X('mean_wind:Q', scale=alt.Scale(domain=(0,120))),
    y=alt.Y('mean_pressure:Q', scale=alt.Scale(domain=(900,1040))),
    color=alt.Color('Nature:N', scale=alt.Scale(domain=('TS','SS','DS','ET','NR')))
).transform_filter(
    interval
).transform_aggregate(
    mean_wind='mean(Wind in knots)',
    mean_pressure='mean(Pressure in millibars)',
    groupby=['Serial_Num','Nature']
).properties(
    width=300,
    height=200
)

timeline | scatter

alt.HConcatChart(...)

In [8]:
# If you have vega_datasets installed

# from vega_datasets import data
# states = alt.topo_feature(data.us_10m.url, feature='states')


## Hurricane tracts (2005)

Pre-filtering this in Pandas since data is so large that don't want to do year 2005 filtering in Altair.

**Interval selection in time bar chart.**

In [12]:
tracts_2005 = tracts.loc[tracts['Season']==2005,:]

states = alt.topo_feature('https://vega.github.io/vega-datasets/data/us-10m.json', 'states')

interval = alt.selection(type='interval', encodings=['x'])

timeline = alt.Chart(tracts_2005).mark_bar().encode(
    x=alt.X('ISO_time:T', timeUnit='yearmonthdate'),
    y='count():Q'
).add_selection(
    interval
).properties(
    width=200,
    height=120
)

background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)

points = alt.Chart(tracts_2005).mark_circle().encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    size='Wind in knots:Q',
    tooltip=['Hurricane Name','Longitude','Latitude']
).transform_filter(
    interval
).project(
    "albersUsa"
).properties(
    width=500,
    height=400
)

timeline | (background + points)

alt.HConcatChart(...)